# YOLOv8 training
This part of the project explores capabilites of the YOLOv8 model.

In [ ]:
! pip install ultralytics

In [ ]:
import os
from ultralytics import YOLO

In [ ]:
BASE_PATH = "C:\\Users\\tlust\\Downloads\\mtsd\\yolov8\\"
detect_path = os.path.join(BASE_PATH, "detect", "dataset.yaml")
classify_path = os.path.join(BASE_PATH, "classify")

DOUBLE_STEP = True

## One-step fully taxonomy detection + classification to 313 classes
Initial experiment to assess the model's default performance across the entire taxonomy. Anticipated to yield suboptimal results due to the extensive number of classes.

In [ ]:
if DOUBLE_STEP == False:
    #model = YOLO('yolov8n.yaml')  # build a new model from YAML
    #model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights
    model = YOLO("yolov8m.pt")  # load a pretrained model
    results = model.train(data=detect_path, epochs=100, imgsz=640, batch=16)
    print(results)

## 2-stage pipeline
Anticipated to yield improved outcomes as a result of decoupling.

### 1. train binary sign detector

In [ ]:
if DOUBLE_STEP:
    #model = YOLO('yolov8n.yaml')                           # build a new model from YAML
    #model = YOLO('yolov8n.yaml').load('yolov8n.pt')        # build from YAML and transfer weights
    model = YOLO('yolov8n.pt')  # load a pretrained model
    results = model.train(data=detect_path, epochs=10, imgsz=640)
    print(results)

### 2. train sign classifier

In [ ]:
if DOUBLE_STEP:
    #model = YOLO('yolov8n-cls.yaml')                           # build a new model from YAML
    #model = YOLO('yolov8n-cls.yaml').load('yolov8n-cls.pt')    # build from YAML and transfer weights
    model = YOLO('yolov8x-cls.pt')  # load a pretrained model
    results = model.train(data=classify_path, epochs=100, imgsz=224, batch=128)
    print(results)

# MM Lab

In [ ]:
! pip install -q supervision==0.13.0
! pip install -q openmim
! mim install -q "mmengine"
! mim install -q "mmcv"
! mim install -q "mmdet"

In [1]:
! git clone https://github.com/Sense-X/Co-DETR.git

Cloning into 'mmdetection'...


In [2]:
%cd Co-DETR
! pip install -e .

c:\Users\tlust\repos\pova-traffic-sign-recognition\notebooks\mmdetection
Obtaining file:///C:/Users/tlust/repos/pova-traffic-sign-recognition/notebooks/mmdetection
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for mmdet


In [ ]:
import cv2
import torch
from mmdet.apis import init_detector, inference_detector
import supervision as sv

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = "C:\\Users\\tlust\\Downloads\\co_dino_5scale_r50_lsj_8xb2_1x_coco.py"
WEIGHTS_PATH = "C:\\Users\\tlust\\Downloads\\co_dino_5scale_r50_lsj_8xb2_1x_coco-69a72d67.pth"

In [ ]:
model = init_detector(CONFIG_PATH, WEIGHTS_PATH, device=DEVICE)

image = cv2.imread("C:\\Users\\tlust\Downloads\\aaa.jpg")
result = inference_detector(model, image)

detections = sv.Detections.from_mmdetection(result)
detections = detections[detections.confidence > 0.3].with_nms()
box_annotator = sv.BoxAnnotator()
annotated_image = box_annotator.annotate(image.copy(), detections)
sv.plot_image(image=annotated_image, size=(10, 10))